In [2]:
import pandas as pd
import numpy as np
import torch

import sys
sys.path.append('../')
from irt import IRT
import os
device ='cuda:0'
train_data_path = "../data/flickr/processed/train/"
test_data_path = "../data/flickr/processed/test/"
irt_models_path = "../data/flickr/evaluation/irt_models/"

In [3]:
irt_model = IRT(interaction=True)
irt_model.load(irt_models_path+'irt_model_interaction_v2')

In [41]:
irt_model.Ma.shape

torch.Size([1280, 80])

In [4]:
data_test = pd.read_csv(test_data_path+"test.csv")
E_test = torch.load(test_data_path+"test_ie.pth", weights_only=True)
U_test = torch.tensor(data_test.worker_id)

In [5]:
irt_model.score(U_test, E_test.to(torch.float32))

{0: {'avg': 0.1177634447813034, 'std_error': 0.03279915824532509},
 1: {'avg': 0.5600583553314209, 'std_error': 0.07339368760585785},
 2: {'avg': 0.3277135193347931, 'std_error': 0.10341393202543259},
 3: {'avg': 0.4700864255428314, 'std_error': 0.0990249440073967},
 4: {'avg': 0.34484416246414185, 'std_error': 0.10139508545398712},
 5: {'avg': 0.4920850396156311, 'std_error': 0.09162384271621704},
 6: {'avg': 0.4223087430000305, 'std_error': 0.08898787945508957},
 7: {'avg': 0.5066087245941162, 'std_error': 0.08657854795455933},
 8: {'avg': 0.19786781072616577, 'std_error': 0.09825316071510315},
 9: {'avg': 0.3794806897640228, 'std_error': 0.11672765016555786},
 10: {'avg': 0.33799731731414795, 'std_error': 0.11337112635374069},
 11: {'avg': 0.6165096759796143, 'std_error': 0.14068792760372162},
 12: {'avg': 0.07545991241931915, 'std_error': 0.03172377124428749},
 13: {'avg': 0.5097498893737793, 'std_error': 0.10177282989025116},
 14: {'avg': 0.3947809338569641, 'std_error': 0.1251197

In [3]:
from diffusion_adapters import StableDiffusionXLPipelineAdapterEmbeddings
from diffusers.utils import load_image

pipe = StableDiffusionXLPipelineAdapterEmbeddings.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")
pipe.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")

C:\Users\Gabriel\.cache\huggingface\hub\models--stabilityai--stable-diffusion-xl-base-1.0\snapshots\462165984030d82259a11f4367a4eed129e94a7b


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
PR_DIR = "../data/flickr/evaluation/diffusion_priors/models/samples/test"

scores = dict.fromkeys(os.listdir(PR_DIR))

for candidate in os.listdir(PR_DIR):
    if "exp2" not in candidate:
        continue
    print(candidate)
    candidate_dir = os.path.join(PR_DIR, candidate)
    user_scores = dict.fromkeys(os.listdir(candidate_dir))
    for user in os.listdir(candidate_dir):
        
        images_dir = os.path.join(candidate_dir, user, "images")
        sdxl_ipadapter_embs = []
        with torch.no_grad():
            for path in os.listdir(images_dir):
                #path = "../data/raw/FLICKR-AES-001/40K/" + unq_img_path
                impath = os.path.join(images_dir, path)
                pil_image = load_image(impath)
                image_emb = pipe.encode_image(pil_image, device="cuda", num_images_per_prompt=1)[0].squeeze()
                sdxl_ipadapter_embs.append(image_emb.cpu())
            sdxl_ipadapter_embs_tensor = torch.stack(sdxl_ipadapter_embs)
            torch.save(sdxl_ipadapter_embs_tensor, f"{images_dir}/../embeddings/sdxl_embeddings.pth")

SPU_80_GS_10.0__NEG__usrthrs_100_exp2
SPU_80_GS_12.0__NEG__usrthrs_100_exp2
SPU_80_GS_14.0__NEG__usrthrs_100_exp2
SPU_80_GS_16.0__NEG__usrthrs_100_exp2
SPU_80_GS_18.0__NEG__usrthrs_100_exp2
SPU_80_GS_20.0__NEG__usrthrs_100_exp2
SPU_80_GS_8.0__NEG__usrthrs_100_exp2


In [34]:
from diffusion_adapters import StableDiffusionXLPipelineAdapterEmbeddings, StableDiffusionPipelineAdapterEmbeddings
from diffusers.utils import load_image

pipe = StableDiffusionXLPipelineAdapterEmbeddings.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")
pipe.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x1024 and 1280x80)